In [1]:
# You will need pandas/numpy to run the first part of the script. Then plotly for the second part. You can install plotly via pip (pip install plotly)
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv(
    r"2020.04.24 153334 test email summary.csv",
    index_col=0)
df['date'] = pd.to_datetime(df['date'])
df['day'] = df['date'].dt.floor('d')
df['hour'] = df['date'].dt.floor('h')
df['company'] = df['sender address'].str.split('@', n = -1, expand = False).apply(lambda x: x[-1])

In [ ]:
df

In [ ]:
email_count_per_ai  = df.groupby('ai').count()
email_count_per_day = df.groupby('day').count()
email_count_per_hour = df.groupby('hour').count()

In [ ]:
max_rows = pd.options.display.max_rows
pd.options.display.max_rows = None
emails_count = df.groupby('sender name').count().sort_values(by='date', ascending=False)
# See https://stackoverflow.com/questions/49574817/fill-a-new-pandas-column-with-row-numbers
emails_count['ranking'] = np.arange(len(emails_count)) + 1
emails_count['emails'] = emails_count['date']
display(emails_count[['ranking', 'emails']].iloc[0:25,:])
pd.options.display.max_rows = 60

In [ ]:
max_rows = pd.options.display.max_rows
pd.options.display.max_rows = None
emails_count_company = df.groupby('company').count().sort_values(by='date', ascending=False)

# See https://stackoverflow.com/questions/49574817/fill-a-new-pandas-column-with-row-numbers
emails_count_company['ranking'] = np.arange(len(emails_count_company)) + 1
emails_count_company['emails'] = emails_count_company['date']
display(emails_count_company[['ranking', 'emails']].iloc[0:20,:])
pd.options.display.max_rows = 60

In [ ]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [ ]:
data = go.Bar(
    x=email_count_per_ai.index, 
    y=email_count_per_ai['date'])
layout = go.Layout(
    xaxis= {'type': 'category'},
    title='Emails per #AI (total {0})'.format(email_count_per_ai['date'].sum()))
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
data = go.Bar(
    x=email_count_per_day.index, 
    y=email_count_per_day['date'])
layout = go.Layout(
    xaxis= {'type': 'category'},
    title='Emails per day (total {0})'.format(email_count_per_ai['date'].sum()))
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
data = go.Bar(
    x=email_count_per_hour.index, 
    y=email_count_per_hour['date'])
layout = go.Layout(
    xaxis= {'type': 'category'},
    title='Emails per hour (total {0})'.format(email_count_per_ai['date'].sum()))
fig = go.Figure(data=data, layout=layout)
fig.show()